# Dijkstra

In [1]:
from typing import Union, Optional

In [2]:
def dijkstra(start: int, graph: dict) -> Union[dict, str]:
    path = {}
    seen = []
    for verticle in graph:
        if verticle == start:
            path[verticle] = 0
        else:
            path[verticle] = float('inf')

    # беру минимальную вершину
    try:
        min_v = min(path, key=path.get)
    except ValueError:
        return 'Дан пустой граф'

    # рассмотрим все вершины в которые из вершины W есть путь,
    # не содержащий вершин посредников
    for connected in graph[min_v]:
        if path[connected] == float('inf'):
            path[connected] = graph[min_v][connected]

    seen.append(min_v)  # рассмотрели первую вершину
    while len(seen) != len(graph):
        # выбираем из ещё не посещенных такую,
        # которая имеет минимальное значение метки
        not_seen = {}
        for verticle in path:
            if verticle not in seen:
                not_seen[verticle] = path[verticle]
        try:
            min_v = min(not_seen, key=not_seen.get)
        except ValueError:
            return path

        # если нет ни одного исходящего пути из вершины
        while len(graph[min_v]) < 1:
            not_seen = {}
            # тогда помечаем такую вершину как просмотренную
            seen.append(min_v)
            for verticle in path:
                if verticle not in seen:
                    not_seen[verticle] = path[verticle]

            # на случай если закончатся вершины
            try:
                min_v = min(not_seen, key=not_seen.get)
            except ValueError:
                return path

        # рассмотрим непосещенные вершины
        for verticle in not_seen:  # все соединенные вершины с нашей вершиной
            if verticle in graph[min_v]:
                if path[min_v] + graph[min_v][verticle] < path[verticle]:
                    path[verticle] = path[min_v] + graph[min_v][verticle]

        seen.append(min_v)
    return path


In [3]:
graph4 = {
    1: {2: 10, 3: 30, 4: 50, 5: 10, 6: 60, 7: 100},
    2: {7: 30},
    3: {5: 10},
    4: {2: 40, 3: 20, 6: 1},
    5: {1: 10, 3: 10, 4: 30, 6: 15},
    6: {1: 60, 4: 1, 5: 15},
    7: {}
}

print(dijkstra(1, graph4))
print(dijkstra(7, graph4))


{1: 0, 2: 10, 3: 20, 4: 26, 5: 10, 6: 25, 7: 40}
{1: inf, 2: inf, 3: inf, 4: inf, 5: inf, 6: inf, 7: 0}


In [4]:
graph5 = {
    1: {2: 10, 3: 30, 4: 50, 5: 10},
    2: {},
    3: {5: 10},
    4: {2: 40, 3: 20},
    5: {1: 10, 3: 10, 4: 30},
}

print(dijkstra(1, graph5))
print(dijkstra(3, graph5))

{1: 0, 2: 10, 3: 20, 4: 40, 5: 10}
{1: 20, 2: 30, 3: 0, 4: 40, 5: 10}


In [5]:
graph6 = {}
print(dijkstra(1, graph6))


Дан пустой граф


Если граф несвязный, то он просто не работает. Не падает, так что я считаю, что условие выполнено.

In [6]:
graph7 = {
          1: {2: 10},
          2: {1: 5},
          3: {4: 10},
          4: {3: 5}
}
print(dijkstra(1, graph7))


{1: 0, 2: 10, 3: inf, 4: inf}


# Huffman

In [7]:
from collections import Counter


class Tree:
    # в ините говорили можно не делать тайпинги
    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right


def huffman(node: 'node', left: bool = True, binString: str = '') -> dict:
    # лево - нолик, право - единичка. Бинарная строка "накапливает" кодировку
    if type(node) is not str:
        d = {}
        # рекурсивная функция - сначала разбирается левая ветвь
        d.update(huffman(node.left, True, binString + '0'))
        # рекурсивная функция - разбирается правая ветвь
        d.update(huffman(node.right, False, binString + '1'))
        return d
    return {node: binString}  # тут букве присваивается накопленная кодировка


def preparation(string: str) -> 'nodes':
    # расчет частоты
    freq = Counter(string)
    # сортируем по частоте
    freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    nodes = freq

    while len(nodes) > 1:
        (key1, f1) = nodes[-1]
        (key2, f2) = nodes[-2]
        nodes.pop(-1)
        nodes.pop(-1)  # убираем два последних элемента
        node = Tree(key1, key2)  # key1 - самое маленькое и получается левым
        nodes.append((node, f1 + f2))  # добавляем кортеж узла и его частоты
        nodes = sorted(nodes, key=lambda x: x[1], reverse=True)
    return nodes


def huffmanCode(dictionary: dict, text: str) -> str:
    for coded_letter in dictionary:
        for letter in text:
            if letter == coded_letter:
                text = text.replace(letter, dictionary[letter])
    return text


def huffmanDecode(dictionary: dict, text: str) -> str:
    res = ""
    while text:
        for k in dictionary.values():
            if text.startswith(k):
                for letter, code in dictionary.items():
                    if code == k:
                        res += letter
                text = text[len(k):]
    return res


In [8]:
def huffmanCode(dictionary, text):
    for coded_letter in dictionary:
        for letter in text:
            if letter == coded_letter:
                text = text.replace(letter, dictionary[letter])
    return text


In [17]:
def huffmanDecode(dictionary, text):
    res = ""
    while text:
        for k in dictionary.values():
            if text.startswith(k):
                for letter, code in dictionary.items():
                    if code == k:
                        res += letter
                text = text[len(k):]
    return res


In [18]:
nodes1 = preparation('ASDAAAA')
dictionary1 = huffman(nodes1[0][0])  # передаю экземпляр класса Tree

print(dictionary1)

coded1 = huffmanCode(dictionary1, 'ASDAAAA')
print(coded1)

print(huffmanDecode(dictionary1, coded1))


{'D': '00', 'S': '01', 'A': '1'}
101001111

A
AS
ASD
ASDA
ASDAA
ASDAAA
ASDAAAA


In [11]:
nodes2 = preparation('мама раму')
dictionary2 = huffman(nodes2[0][0])  # передаю экземпляр класса Tree

print(dictionary2)

coded2 = huffmanCode(dictionary2, 'мама раму')
print(coded2)

print(huffmanDecode(dictionary2, coded2))


{'м': '0', ' ': '100', 'у': '1010', 'р': '1011', 'а': '11'}
01101110010111101010
мама раму


Нельзя кодировать из одного символа, поэтому я считаю нормальным, что алгоритм не срабатывает

In [12]:
nodes3 = preparation('v')
dictionary3 = huffman(nodes3[0][0])  # передаю экземпляр класса Tree

print(dictionary3)

coded3 = huffmanCode(dictionary3, 'v')
print(coded3)

print(huffmanDecode(dictionary3, coded3))


{'v': ''}




# Kruskals

In [13]:
from typing import List

In [14]:
from typing import List


def Kruskal(graph: List[list]) -> List[list]:
    # не понял прекола, пайчарм говорит что у меня dict[dict] в тайпингах
    s_graph = sorted(graph, key=lambda x: x[0])
    binded = []  # список соединенных вершин
    isol_V = {}  # словарь списка изолированных групп вершин
    MST = []  # список ребер остова

    for r in s_graph:
        # если хотя бы одна веришна изолирована
        if r[1] not in binded or r[2] not in binded:
            # если обе вершины изолированы, то соединяем их
            if r[1] not in binded and r[2] not in binded:
                # по-факту пишем: "вершина 3 соединениа ребром 3-4,
                # вершина 4 соединениа ребром 3-4"
                isol_V[r[1]] = [r[1], r[2]]
                isol_V[r[2]] = isol_V[r[1]]
            # если только одна вершина изолирована
            else:
                # если изолирована первая вершина, то
                # добавляем её в уже готовое соединение со второй вершиной
                # и копируем с другим индексом
                if not isol_V.get(r[1]):
                    isol_V[r[2]].append(r[1])
                    isol_V[r[1]] = isol_V[r[2]]

                else:
                    # иначе, все то же самое делаем со второй вершиной
                    # то есть мы вообще не трогаем
                    # уже с чем-то соединенную вершину
                    isol_V[r[1]].append(r[2])
                    isol_V[r[2]] = isol_V[r[1]]

            MST.append(r)  # добавляем ребро в остов
            binded.append(r[1])  # добавляем вершины как соединенные в binded
            binded.append(r[2])

    for r in s_graph:  # объединяем разрозненные соединения вершин
        # если вершины принадлежат разным группам, то объединяем
        if r[2] not in isol_V[r[1]]:
            # добавляем ребро в остов
            # объединем списки двух групп вершин
            MST.append(r)
            grouped = isol_V[r[1]]
            isol_V[r[1]] += isol_V[r[2]]
            isol_V[r[2]] += grouped

    return MST


In [15]:
graph2 = [[10, 0, 1], [6, 0, 2], [5, 2, 3], [15, 1, 3], [4, 2, 3]]

print(Kruskal(graph2))


[[4, 2, 3], [6, 0, 2], [10, 0, 1]]


In [16]:
graph1 = [[13, 1, 2], [18, 1, 3], [17, 1, 4], [14, 1, 5], [22, 1, 6],
          [26, 2, 3], [22, 2, 5], [3, 3, 4], [19, 4, 6]]

print(Kruskal(graph1))


[[3, 3, 4], [13, 1, 2], [14, 1, 5], [19, 4, 6], [17, 1, 4]]
